## 라이브러리 로드

In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져옵니다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱합니다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러옵니다.
import pandas as pd

## 서울시 코로나19 발생동향
* [코로나19](http://www.seoul.go.kr/coronaV/coronaStatus.do)

## requests 로 html 문서 받아오기

In [2]:
# 크롤링 할 사이트
base_url = "http://www.seoul.go.kr/coronaV/coronaStatus.do"
response = requests.get( base_url )
response

<Response [200]>

In [3]:
if response.status_code == 200:
    soup = bs(response.text, 'html.parser')

str(soup)[:1000]

'<!-- ######################################################################## -->\n<!-- \r\n\tmenu name:\t\t코로나 바이러스 현황 단독 페이지\r\n-->\n<!-- ######################################################################## -->\n<!DOCTYPE html>\n\n<html lang="ko">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport">\n<title>코로나19</title>\n<meta content="코로나19" name="description"/>\n<meta content="website" property="og:type"/>\n<meta content="코로나19 - 2주간의 \'잠시 멈춤\' 캠페인" property="og:title"/>\n<meta content="코로나19를 멈추기 위해 우리도 잠시 멈춰요." property="og:description"/>\n<meta content="https://www.seoul.go.kr/coronaV/coronaStatus.do" property="og:url"/>\n<meta content="http://www.seoul.go.kr/res_newseoul/images/corona/pic_facebook_20200304.jpg" property="og:image"/>\n<meta content="summary" name="twitter:card"/>\n<meta content="코로나19 - 2주간의 \'잠시 멈춤\' 캠페인" name="twitter:title"/>\n<meta content="코로나19

## 서울 확진자 현황

In [4]:
#move-cont1 > div:nth-child(2) > div

#move-cont1 > div:nth-child(2) > div
cont_page = soup.select("#move-cont1 > div > div.cont-page-wrap > div.cont-page")
len(cont_page)

2

In [13]:
trs_ex =  cont_page[0].select("tr")
trs_ex[:3]

[<tr><th scope="col">연번</th><th scope="col">환자</th><th scope="col">확진일</th><th scope="col">성별(출생년)</th><th scope="col">거주지</th><th scope="col">여행력</th><th scope="col">접촉력</th><th scope="col">조치사항</th></tr>,
 <tr><th scope="row">156</th><td data-tit="환자 번호" scope="col">확인중</td><td data-tit="확진일">3.9.</td><td data-tit="성별(출생년)">여('85)</td><td data-tit="거주지">구로구</td><td data-tit="여행력">-</td><td data-tit="접촉력">#7265 접촉(추정)</td><td data-tit="조치사항">서남병원</td></tr>,
 <tr><th scope="row">155</th><td data-tit="환자 번호" scope="col">7444</td><td data-tit="확진일">3.9.</td><td data-tit="성별(출생년)">여('66)</td><td data-tit="거주지">강서구</td><td data-tit="여행력">-</td><td data-tit="접촉력">#7265 접촉(추정)</td><td data-tit="조치사항">서남병원</td></tr>]

## 행 데이터 리스트로 만들기

In [37]:
def parse_tr(trs):
    table = []
    for tr in trs:
        tds = tr.select("td")
        if len(tds) > 0 :
            row = []
            # 연번
            number = tr.select("th")[0].get_text()
            row.append(number)
            # 연번 외 데이터
            for td in tds:
                val = td.get_text()
                row.append(val)
            table.append(row)
    return table

parse_tr(trs_ex[:3])

[['156', '확인중', '3.9.', "여('85)", '구로구', '-', '#7265 접촉(추정)', '서남병원'],
 ['155', '7444', '3.9.', "여('66)", '강서구', '-', '#7265 접촉(추정)', '서남병원']]

## 페이지마다 가져오기

In [38]:
#move-cont1 > div:nth-child(2) > table
#move-cont1 > div:nth-child(2) > table.tstyle05.tstyleP > thead > tr > th:nth-child(1)
#cont-page2
table_result = []
for page in cont_page:
    trs = page.select("tr")
    result = parse_tr(trs)
    table_result.extend(result)

table_result

[['156', '확인중', '3.9.', "여('85)", '구로구', '-', '#7265 접촉(추정)', '서남병원'],
 ['155', '7444', '3.9.', "여('66)", '강서구', '-', '#7265 접촉(추정)', '서남병원'],
 ['154', '확인중', '3.9.', "여('69)", '구로구', '-', '#7265 접촉(추정)', '서울의료원'],
 ['153', '확인중', '3.9.', "여('70)", '구로구', '-', '#7265 접촉(추정)', '서울의료원'],
 ['152', '확인중', '3.9.', "여('74)", '광명', '-', '#7265 접촉(추정)', '타시도이관'],
 ['151', '확인중', '3.9.', "여('71)", '광명', '-', '#7265 접촉(추정)', '서남병원'],
 ['150', '7469', '3.9.', "여('60)", '강서구', '-', '#7265 접촉(추정)', '서남병원'],
 ['149', '확인중', '3.9.', "여('67)", '부천', '-', '#7265 접촉(추정)', '타시도이관'],
 ['148', '7480', '3.9.', "남('65)", '구로구', '-', '콜센터직원 접촉', '서울의료원'],
 ['147', '확인중', '3.9.', "남('98)", '구로구', '-', '콜센터직원 접촉', '서울의료원'],
 ['146', '확인중', '3.9.', "여('69)", '구로구', '-', '#7265 접촉(추정)', '서남병원'],
 ['145', '확인중', '3.9.', "남('75)", '구로구', '-', '#7265 접촉(추정)', '보라매병원'],
 ['144', '7502', '3.9.', "여('71)", '강서구', '-', '#7265 접촉(추정)', '서남병원'],
 ['143', '확인중', '3.9.', "남('77)", '양천구', '-', '#7265 접촉(추정)', '보라매병원'],
 ['14

## 컬럼명 만들기

In [39]:
# 컬럼명 만들기
cols = trs_ex[0].select("th")
cols

[<th scope="col">연번</th>,
 <th scope="col">환자</th>,
 <th scope="col">확진일</th>,
 <th scope="col">성별(출생년)</th>,
 <th scope="col">거주지</th>,
 <th scope="col">여행력</th>,
 <th scope="col">접촉력</th>,
 <th scope="col">조치사항</th>]

In [40]:
col_name = []
for col in cols:
    col_name.append(col.get_text())
col_name

['연번', '환자', '확진일', '성별(출생년)', '거주지', '여행력', '접촉력', '조치사항']

## 데이터프레임으로 만들기

In [41]:
df = pd.DataFrame(table_result, columns=col_name)
print(df.shape)
df.head()

(156, 8)


,연번,환자,확진일,성별(출생년),거주지,여행력,접촉력,조치사항
0,156,확인중,3.9.,여('85),구로구,-,#7265 접촉(추정),서남병원
1,155,7444,3.9.,여('66),강서구,-,#7265 접촉(추정),서남병원
2,154,확인중,3.9.,여('69),구로구,-,#7265 접촉(추정),서울의료원
3,153,확인중,3.9.,여('70),구로구,-,#7265 접촉(추정),서울의료원
4,152,확인중,3.9.,여('74),광명,-,#7265 접촉(추정),타시도이관


## 확진일로 정렬하고 CSV 파일로 저장하기

In [46]:
df = df.sort_values(by=["연번", "확진일"], ascending=False)
df.to_csv("covid-19-seoul.csv", index=False)

## CSV 파일 읽어오기

In [47]:
pd.read_csv("covid-19-seoul.csv").head(10)

,연번,환자,확진일,성별(출생년),거주지,여행력,접촉력,조치사항
0,99,5377,3.1.,여('96),동대문구,-,대구 방문,서북병원
1,98,4743,3.2.,남('66),서초구,-,확인중,서울의료원
2,97,4709,3.1.,남('85),양천구,-,타시도 확진자 접촉,서울의료원
3,96,4645,3.2.,남('82),도봉구,-,확인중,서울의료원
4,95,4634,3.2.,여('99),성북구,-,성동구 주상복합 관련,서울의료원
5,94,4558,3.2.,여('69),성북구,-,성동구 주상복합 관련,서울의료원
6,93,4517,3.2.,여('69),강남구,-,부동산중개법인관련,서울의료원
7,92,4224,3.2.,남('00),영등포구,-,성동구 주상복합 관련,서북병원
8,91,4174,2.29.,남('72),영등포구,-,여의도 건설현장관련,서울의료원
9,90,3815,3.1.,남('69),영등포구,-,성동구 주상복합 관련,서울의료원


In [48]:
df["접촉력"].value_counts()

확인중              22
#7265 접촉(추정)     18
해외 접촉            13
성동구 주상복합 관련      13
타시도 확진자 접촉       10
은평성모병원관련          7
은평성모병원 관련         6
대구 방문             5
#7103 접촉          4
타시도 확진자 접촉자       3
콜센터직원 접촉          3
#6 접촉자            3
대구 확진자 접촉         3
#6832 접촉자         2
타시도 확진자 접촉        2
명성교회 관련           2
신천지대구교회 관련        2
#780 접촉자          2
#794 접촉자          2
해외접촉              2
쇼핑몰 관련            1
#29접촉자            1
대구 거주             1
#6접촉자(추정)         1
#6727 첩촉          1
확잍중               1
#2621번 접촉자        1
대구 거주자            1
#30접촉자(추정)        1
은평성모병원 연관성 검토     1
#83접촉자            1
#5 접촉자            1
#56접촉자            1
#935 접촉자          1
해외 접촉(추정)         1
#5592 접촉(추정)      1
#3 접촉자            1
경찰병원관련            1
대구 방문(명성교회)       1
#136접촉자           1
부동산 중개 법인관련       1
-                 1
여의도 건설현장관련        1
#6921접촉자          1
여의도 건설현장 관련       1
#6937 접촉자         1
#780 가족(아들)       1
부동산중개법인관련         1
#7300 접촉          1
서울재생병원 관련         1


In [45]:
df["조치사항"].value_counts()

서울의료원          43
서남병원           22
보라매병원          17
서울의료원(퇴원)      10
국립중앙의료원         8
서북병원            8
서울대학교병원(퇴원)     7
국립중앙의료원(퇴원)     7
타시도이관           5
서울대학교병원         5
중앙대학교병원         3
삼육서울병원          2
고대구로병원          2
순천향서울병원         2
한일병원            2
보라매병원(퇴원)       2
인제대상계백병원        2
타시도 이관          2
상계백병원           1
타시도이관(퇴원)       1
서울백병원           1
한양대학교병원         1
중앙대학교병원(퇴원)     1
한일병원(퇴원)        1
한양대학교병원(퇴원)     1
Name: 조치사항, dtype: int64